# Getting Objects and Boundaries from Images

In [1]:
# pip install --upgrade google-cloud-vision
#pip install opencv-python

In [2]:
import numpy as np
import pandas as pd

#import OpenCV to annotate images with shapes
import cv2

# google cloud SDK
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

In [3]:
# Code Reference From: https://github.com/googleapis/google-cloud-python/issues/5349
# need this in order to access API - enter your JSON file in the path after you have created your service account

from google.oauth2 import service_account
credentials = service_account.Credentials. from_service_account_file("C:/Users/Erik/Desktop/flood_project/flooddepths-0158d30f7869.json")

### Path to folder with images

In [4]:
# change the path to the current folder where the images that need to be annotated are located
path_to_images = 'C:/Users/Erik/Desktop/add_photos/add_flood/'

### Path to folder where the cropped photo should go

In [5]:
path_to_cropped_images = 'C:/Users/Erik/Desktop/add_photos/cropped_add_flood/'

### List of Images

In [6]:
# a list of all of my picture filenames that have floods & cars
# with help from https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory

flood_list = [f for f in os.listdir(path_to_images) if os.path.isfile(os.path.join(path_to_images, f))]

In [7]:
objects_to_crop_around = ['Car','Van','Truck','Boat','Toy vehicle']

## Cropping images to use in NN

In [8]:
def crop_flooded_objects_boundary(file_list):
    
    file_dict = {}
    object_dict = {}
    
    # creating a tenth and cycle counter to output progress of function
    tenth_counter = 0
    cycle_counter = 1
    
    #looping though each image in the list submitted to the function
    for file in file_list:
    
        #need to have google vision credentials saved to credentials
        client = vision.ImageAnnotatorClient(credentials=credentials)

        # path to the images that need to be cropped
        with open(path_to_images + file, 'rb') as image_file:
            content = image_file.read()
        image = vision.types.Image(content=content)
        
        #same path just using OpenCV to get image shape and will use to save the cropped images later
        im_cv2 = cv2.imread(path_to_images + file)
        height, width, color = im_cv2.shape

        #Using Google vision to actually find objects in the image
        objects = client.object_localization(image=image).localized_object_annotations
        
        object_list = []
        
        tenth_counter += 1
        
        # printing out the fraction of the images done to keep track of function progress
        if int(round(len(flood_list),-1)/10) <= tenth_counter:
            print(f'{cycle_counter}/10 done')
            cycle_counter += 1
            tenth_counter = 1
        
        # creating an item counter to allow the cropping and the saving of multiple same objects from one photo
        item_counter = 1
        
        #looping through each of the objects Google vision found in the image
        for object_ in objects:
            # ignoring all objects that don't have to do with the cars in the image
            if object_.name in objects_to_crop_around:
                vertex_dict = {}

                #need to make sure the normalized vertex are multipled by the corresponding image distance so the vertex are in pixels counts
                for index,vertex in enumerate(object_.bounding_poly.normalized_vertices):
                    vertex_dict[f'vertex_{index}'] = [int(width*vertex.x),int(height*vertex.y)]
                object_dict[object_.name] = vertex_dict
            
                # Cropping the image around the vertices of the object
                
                # https://www.life2coding.com/cropping-polygon-or-non-rectangular-region-from-image-using-opencv-python/
                # https://stackoverflow.com/questions/48301186/cropping-concave-polygon-from-image-using-opencv-python
                
                mask = np.zeros(im_cv2.shape[:2], np.uint8)
                points = np.array([object_dict[object_.name]['vertex_0'],
                                   object_dict[object_.name]['vertex_1'],
                                   object_dict[object_.name]['vertex_2'],
                                   object_dict[object_.name]['vertex_3']])
            
                #creating the bounding rectangle from the object vertices
                rect = cv2.boundingRect(points)
                x,y,w,h = rect
                
                # cropping the image using OpenCV and the dimentions of the bounding rectangle
                cropped = im_cv2[y:y+h, x:x+w].copy()
            
                #savig the cropped image using OpenCV. Image name has cropped_(object)_ added to the front
                cv2.imwrite(path_to_cropped_images + 'cropped_' + object_.name + str(item_counter) + '_' + file, cropped)
                
                #adding 1 to the item counter so multiple cars can be cropped and then saved from the same image
                item_counter += 1
                
        file_dict[file] = object_dict
        
    return file_dict

In [9]:
#cropping all imaged in the flood_list. Run with a semi colon to suppress the dictionary from showing upon completion
object_boundary_dict = crop_flooded_objects_boundary(flood_list)

1/10 done
2/10 done
3/10 done
4/10 done
5/10 done
6/10 done
7/10 done
8/10 done
9/10 done
10/10 done
11/10 done


## Function to get just get the object lables and their boundaries

In [27]:
# Just gets the objects and their boundaries.
def determine_objects_boundary(file_list):
    
    file_dict = {}
    object_dict = {}
    
    #looping though each image in the list submitted to the function
    for file in file_list:
    
        #need to have google vision credentials saved to credentials
        client = vision.ImageAnnotatorClient(credentials=credentials)

        # path to the images that need to be cropped
#         with open('C:/Users/Erik/Desktop/flood_project/images/' + file, 'rb') as image_file:
        with open(path_to_images + file, 'rb') as image_file:
            content = image_file.read()
        image = vision.types.Image(content=content)
        
        #same path just using OpenCV to get image shape
#         im_cv2 = cv2.imread('C:/Users/Erik/Desktop/flood_project/images/' + file)
        im_cv2 = cv2.imread(path_to_images + file)
        height, width, color = im_cv2.shape

        #Using Google vision to actually find objects in the image
        objects = client.object_localization(image=image).localized_object_annotations
        
        object_list = []
        
        #for each object in the image, find the 4 corners of the bounding rectangle and save them in a dict
        for object_ in objects:
            vertex_dict = {}

            #need to make sure the normalized vertex are multipled by the corresponding image distance so the vertex are in pixels counts
            for index,vertex in enumerate(object_.bounding_poly.normalized_vertices):
                vertex_dict[f'vertex_{index}'] = [int(width*vertex.x),int(height*vertex.y)]
            object_dict[object_.name] = vertex_dict
            
        file_dict[file] = object_dict

            
    return file_dict